根據把首頁往下滑，發現網站會對server送一個GET請求下一個30篇文章
- 2018(G) https://ithelp.ithome.com.tw/articles/10203216
- 2019(G) https://tlyu0419.github.io/2019/04/06/Crawl-Dcard/
- 2020 https://blog.jiatool.com/posts/dcard_api_v2/

### API root

- 看板資訊 http://dcard.tw/_api/forums
- 文章資訊 http://dcard.tw/_api/forums/{看板名稱}/posts
- 文章內文	http://dcard.tw/_api/posts/{文章編號}
- 文章內引用連結	http://dcard.tw/_api/posts/{文章編號}/links
- 文章內留言	http://dcard.tw/_api/posts/{文章編號}/comments

### API wirte
- 文章數量 https://www.dcard.tw/_api/posts?limit=100
- 文章分頁 http://dcard.tw/_api/posts/?before=文章id (before 回傳由新到舊 after舊到新)
- 文章留言 http://dcard.tw/_api/posts/文章id/comments
- 留言分頁 http://dcard.tw/_api/posts/文章id/comments?after=floor數字
- 熱門文章 https://dcard.tw/_api/posts?popular=true

### others
文章資訊(meta) 與 文章內留言 預設使用熱門度 (popularity) 作為排序，而且一次請求 (request) 中只回應 30 筆。
* 想要讓這兩項使用 時間 排序，可在 GET 參數中加入 popular=false
* 欲取得更多的 文章資訊(meta)，可以使用 before={某文章編號}來獲得早於 #{某文章編號} 的另外 30 筆 資訊。
* 欲取得更多的 文章內留言，可以使用 after={某樓層} 來獲得大於 #{某樓層} 的另外 30 筆 留言。

In [1]:
import requests
import json
import pandas as pd    # 引用套件並縮寫為 pd
from bs4 import BeautifulSoup
# https://www.dcard.tw/f/makeup

GET 美妝版首頁，Use BeautifulSoup with html, wrtie a txt

 

In [2]:
test = open("data/test.txt","w",encoding='UTF-8')

p = requests.Session()                            # 把這一次的request都算在同一個session裡
url=requests.get("https://www.dcard.tw/f/makeup") # 請求的網址
soup = BeautifulSoup(url.text,"html.parser")
sel = soup.select("h2.tgn9uw-2 a")
a=[]
#for link in sel:
#    a.append(link["href"])
    #print('https://www.dcard.tw'+link.get('href'))

In [3]:
# slope the title

def Catch(num):
    try:
        url = 'https://www.dcard.tw/_api/posts/'+str(num)  #api post with ID
        resq = requests.get(url)
        rejs = resq.json()
        return(pd.DataFrame(
            data=
            [{'id':rejs['id'],
              'title':rejs['title'],
              'content':rejs['content'],
              'excerpt':rejs['excerpt'],
              'createdAt':rejs['createdAt'],
              'updatedAt': rejs['updatedAt' ],
              'forumName':rejs['forumName'],
              'forumAlias':rejs['forumAlias'],
              'gender':rejs['gender'],
              'topics':rejs['topics']}],
            columns=['id', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )
    except:
        print('data error...',i,'J:',j)
        return(pd.DataFrame(
            data=
            [{'id':"",
              'title':"",
              'content':"",
              'excerpt':"",
              'createdAt':"",
              'updatedAt': "",
              'forumName':"",
              'forumAlias':"",
              'gender':"",
              'topics':""}],
            columns=['id', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )

In [4]:
# KeyError: 'id'
# Oh！文章不見了 這篇文章已被作者刪除。 按這裡 看更多文章吧
# https://www.dcard.tw/f/makeup/p/235079399
#Catch(235079399)

In [10]:
import random
import time

#url = "http://dcard.tw/_api/forums/makeup/posts?popular=false&limit=100"
# 1w : 234227720
# 2w : 233254532
url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=false&limit=100&before=233254532" 
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df = df.append(Catch(rejs[0]['id']),ignore_index=True)   #0 - 29

delay_choices = [1, 3, 5]  #延遲的秒數
delay = random.choice(delay_choices) 

print(str(int(df['id'].tail(1))) )

233254460


In [ ]:
# 大工程 ok
count = 0

for j in range(100):
    last = str(int(df['id'].tail(1))) # 找出爬出資料的最後一筆ID
    print(last)
    url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=false&limit=100&before=" + last
    resq = requests.get(url)
    rejs = resq.json()
    #rejs[57]['id']
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        count = count+1
        print('data reading...',count,'id:',rejs[i]['title'])
    delay = random.choice(delay_choices)
    time.sleep(delay)  
df.to_csv('data/Op_Dcard_markeup.csv', index=False)
print(df.shape)
df

233254460
data reading... 1 id: 有人跟我一樣不喜歡臉上黏黏的嗎？
data reading... 2 id: Lancome 274家族都上市啦～～～
data reading... 3 id: 求 雅詩蘭黛 母親節DM
data reading... 4 id: 頰彩｜MAC 持色奶凍腮紅 -6色-
data reading... 5 id: 請益水霧眉 這樣是正常的嗎？
data reading... 6 id: 好美～試色 Innisfree 櫻花🌸眼影盤 補充影片～
data reading... 7 id: 生理期霧眉
data reading... 8 id: #體驗箱  我平常怎麼用眉粉畫眉毛、我唯一使用的眉粉 KATE
data reading... 9 id: ＃ 史上最高CP值！泰美麗的秘密武器💰HEART DANCE心動唇萃讓你咚咚咚❤️
data reading... 10 id: 請益混油肌粉底
data reading... 11 id: #情報 pony持久無瑕氣墊粉餅（金盒）第二件1元
data reading... 12 id: #分享🔥 植村秀 時尚大師九色眼影盤-櫻花燦金
data reading... 13 id: #問 Moonshot 氣墊粉餅
data reading... 14 id: ＃底妝實測  蘭蔻全新【超極光精華水粉底】 #極光粉底
data reading... 15 id: #精華液實測  蘭蔻美肌金三角
data reading... 16 id: Dcard最燒美白精華🔥〖極透光亮白淡斑精華〗14天養成淨白透亮肌✨✨Neogence 霓淨思再送你刮刮卡試手氣！
data reading... 17 id: #試色 Etude House新九宮格沙漠城堡
data reading... 18 id: 潤色防曬選手
data reading... 19 id: ＃分享 Remiina/Neogence/Love Dear洗面乳
data reading... 20 id: 乾癢魚鱗腿の愛 💙妮維雅極潤修護乳液
data reading... 21 id: #問 類似的唇膏
data reading... 22 id: 眉毛畫法分享+超美開架橘色腮紅🍊
data reading

data reading... 194 id: 徵板橋 免費美睫練習麻豆
data reading... 195 id: 有人用過碧歐斯這款洗面乳嗎🙏🙏
data reading... 196 id: 進攻超冷門開架彩妝神秘地帶！199十色眼影🔥
data reading... 197 id: innisfree和etude house 眉筆顏色選擇(?
data reading... 198 id: #二更新#圖#文非常非常長 分享閉鎖+悶痘爛臉心路歷程
data reading... 199 id: GA524-GA206的姐妹❤️
data reading... 200 id: #詢問 Kiehl's激光極淨白縮時安瓶
233233530
data reading... 201 id: #分享 重回透亮肌的新歡舊愛們💕 (Simple/艾詩緹/ORBIS)
data reading... 202 id: 痘膚潤價錢
data reading... 203 id: Celvoke莫蘭迪配色眼影分享ex01
data reading... 204 id: 🍒超顯白的櫻桃色🍒
data reading... 205 id: 相見恨晚的innisfree綠茶籽保濕精華
data reading... 206 id: #體驗箱 好自然不失手 KATE 3D造型眉彩餅
data reading... 207 id: 生日禮物
data reading... 208 id: #分享 會回購的美白面膜🤍
data reading... 209 id: isoi 天然玫瑰保濕護唇膏 💋❗
data reading... 210 id: 開架粉餅遮瑕實測👛 #請益遮瑕保濕粉餅選擇
data reading... 211 id: 只擦隔離乳
data reading... 212 id: 熟透的棕色櫻桃🍒
data reading... 213 id: 開架 秋意濃唇膏🍂🍁
data reading... 214 id: 凍膜保養分享
data reading... 215 id: ＃控油實測 貝膚黛瑪 平衡控油精華露＆高效保濕水凝乳
data reading... 216 id: #請益 眼影打底膏和遮瑕
data reading... 217 id: 揪團 bleachlond

data reading... 397 id: 粉餅比較和推薦  資生堂 vs make up for ever vs covermark  蜜粉 妝前精華推薦~
data reading... 398 id: #化妝包分享 輕便帶出門也可以有完整妝容
data reading... 399 id: 紋繡學院選擇
data reading... 400 id: 3ce 眼影盤overtake⋯
233211256
data reading... 401 id: 有人用過MUF的唇拓嗎？
data reading... 402 id: 乾肌的專櫃粉底液選擇
data reading... 403 id: 男生用的專櫃精華選擇！？（有選手）
data reading... 404 id: Belif產品心得分享
data reading... 405 id: 曬眉毛(⁎⁍̴̛ᴗ⁍̴̛⁎)
data reading... 406 id: 粉底刷請益（有選手）
data reading... 407 id: （更）猶豫症發作-Canmake腮紅選擇
data reading... 408 id: 請問GA會員
data reading... 409 id: 轉：#情報 蝦皮高應大專賣店8折
data reading... 410 id: Blackrouge 唇釉快給我去買
data reading... 411 id: #問 高雄哪裡還有賣思麗安
data reading... 412 id: ＃小小更 皮膚科醫生說不要擦它
data reading... 413 id: 萬用的I'M MEME氣墊唇筆006
data reading... 414 id: 美睫教學推薦店家
data reading... 415 id: LED燈光化妝鏡 5名選手
data reading... 416 id: 轉：大家知道coni康倪嗎？
data reading... 417 id: 碎的不是全新高光，是我的心💔
data reading... 418 id: 關於蘭蔻櫃哥選粉底色號
data reading... 419 id: NARS激情過後12色眼彩盤 試色分享🔥🔥🔥
data reading... 420 id: 蝦皮ig的保養步驟正確嗎
data reading... 4

233186302
data reading... 601 id: 強迫推銷？填問卷？
data reading... 602 id: 繡眉後這樣的眉型算成功嗎？
data reading... 603 id: #實戴分享 小直徑隱眼OPT
data reading... 604 id: 螢光眼影
data reading... 605 id: 超巨大粉刺😱
data reading... 606 id: #分享 近期購入的超愛彩妝 專櫃/開架
data reading... 607 id: 遮痣的煩惱
data reading... 608 id: @cosme store bye bye sale
data reading... 609 id: (請益)消毒耳環的替代方案
data reading... 610 id: 植村秀新版定裝噴霧🧡
data reading... 611 id: #請益 包裝好看的霧面紅色系口紅
data reading... 612 id: #從頭到腳的懶人保養法# #不專業分享#
data reading... 613 id: 募集口罩不掉妝案例 分享樓(๑•̀ㅂ•́)و✧
data reading... 614 id: 好用開架唇膏！Solone唇膏唇釉分享！#001 #007 #009 #012 #107 #111
data reading... 615 id: solone迷你唇膏102
data reading... 616 id: Za美白隔離霜
data reading... 617 id: #請益  精華挑選困難
data reading... 618 id: #情報 #屈臣氏 醫美金評賞全部品牌第二件5折
data reading... 619 id: 乾癬患者適合的保養品
data reading... 620 id: 愛的迫降女二徐丹色 植村秀RD153#罌粟紅
data reading... 621 id: 素顏唇膏幫選🥺
data reading... 622 id: #開箱 M.A.C.春櫻漫舞系列🌸
data reading... 623 id: Peripera好長一盤眼影2
data reading... 624 id: 這次來香奈兒敗家了！
data reading... 625 id: 紋眉推薦


data reading... 805 id: 最近好物研究室買的BB美妝蛋 v.s. 韓牌美妝蛋
data reading... 806 id: 耳根子軟的雅詩蘭黛護膚會分享
data reading... 807 id: #問 契爾氏激光淡斑精華
data reading... 808 id: 眉毛求批 QQQQQ
data reading... 809 id: #試色 4U2 擁有好看霧面唇💜
data reading... 810 id: I’m meme 我愛水凝雪酪氣墊粉餅✨🥰
data reading... 811 id: #文長圖多 油痘肌洗面乳選擇障礙（內有選手）
data reading... 812 id: 背部痘疤-台中醫美診所推薦
data reading... 813 id: 這支是什麼😂
data reading... 814 id: 【問】過年七折colourpop有人收到貨了嗎？
data reading... 815 id: 皮膚進化史（內有皮膚照、粉刺照
data reading... 816 id: #問 用完保濕機能水臉好像乾乾的
data reading... 817 id: APIEU積雪草乳霜
data reading... 818 id: 愛的破碎( ´･ᴗ･` )
data reading... 819 id: Estée Lauder 超浮誇禮物包裝🎁
data reading... 820 id: 請益 這是什麼雙眼皮膠
data reading... 821 id: 非常時期只能露眼睛👀
data reading... 822 id: 我的眉毛步驟分享！含眉頭刷刷影片😍
data reading... 823 id: 臉部保養心得 （無過多修辭，白話文）
data reading... 824 id: #試色 Visee新版四色眼影！
data reading... 825 id: BeautyBlender組合開箱！
data reading... 826 id: Spa按摩機
data reading... 827 id: 開架精華液請益
data reading... 828 id: #請益 新手腮紅、修容打亮選擇
data reading... 829 id: 植村秀定妝噴霧
data reading

...

data reading... 6292 id: 不要再被騙了
data reading... 6293 id: #情報 露華濃唇膏組
data reading... 6294 id: #黑特 YSL 奢華緞面水光染唇釉 #610
data reading... 6295 id: #詢問 蝦皮真假
data reading... 6296 id: 請益 關於妝前乳
data reading... 6297 id: 近期彩妝分享🌟
data reading... 6298 id: 救星請進來😭😭🙏
data reading... 6299 id: 幫媽媽挑化妝品
data reading... 6300 id: 秋冬補妝粉餅 選擇
232534694
data reading... 6301 id: #吐嘈 Pony Effect渲霧奶油唇膏
data reading... 6302 id: #問 這是哪盤眼影盤？
data reading... 6303 id: #問植村秀55刷類似款
data reading... 6304 id: 大家出國都會買保養品帶回家嗎？🤣 #分享文
data reading... 6305 id: 原來我也有死亡芭比粉
data reading... 6306 id: 秋冬唇膏疊擦絕配分享🍂✨
data reading... 6307 id: #開箱 1224更 Pony腦粉每次買Pony Effect唇彩都會踩雷這次也不意外：渲霧奶油唇膏試色+心得分享
data reading... 6308 id: #問 有人還買得到嗎？
data reading... 6309 id: #問 有人被廣告燒到的嗎｜ω・)
data reading... 6310 id: #圖 #底妝 讓粉底更持久的妮維雅霜
data reading... 6311 id: YSL 奢華緞面水光染唇釉 #610 迷漾曖昧 泰奶色
data reading... 6312 id: 開箱FZC唇釉
data reading... 6313 id: 週年慶買到剁手系列（保養品）
data reading... 6314 id: 簡單分享 medicube遮瑕心得by粉紅廢宅
data reading... 6315 id: #澳洲保養 #Bio-E 檸檬酵素飲
data 

data reading... 6486 id: #隱眼 低調美~花兒Kukka
data reading... 6487 id: 2019空凹賞-洗卸篇
data reading... 6488 id: #請益 小棕眼霜會不會長肉芽
data reading... 6489 id: 美上美皮膚科診所
data reading... 6490 id: fit me vs revlon粉底液👀
data reading... 6491 id: #請益 除Solone外的平價眼影推薦
data reading... 6492 id: #證件妝容#分享#不花錢 自製證件照OK🉐！
data reading... 6493 id: #分享 最近喜歡的妝容和愛爆的秋天口紅🥀
data reading... 6494 id: #圖多分享 近期的眼妝
data reading... 6495 id: 凹凹賞(✪ω✪)
data reading... 6496 id: 戀愛魔鏡單色眼影打不開
data reading... 6497 id: 韓國 Romand 果汁唇釉 新色分享 #10 #11 #13
data reading... 6498 id: 🔥MAKE UP FOR EVER 刷具組特價
data reading... 6499 id: Bobbi brown 迷戀輕吻唇露用到抓狂...求解
data reading... 6500 id: #分享 我在英國愛用什麼 🇬🇧
232510537
data reading... 6501 id: 雅詩蘭黛-聖誕禮盒開箱、試色(法國版）
data reading... 6502 id: 請問有人使用過藏元美人的產品嗎？
data reading... 6503 id: 雙眼皮變單眼皮
data reading... 6504 id: 想請問一下這家網拍真假
data reading... 6505 id: 三天兩夜極簡化妝包
data reading... 6506 id: #求推薦 透明眉膠
data reading... 6507 id: #問 霓淨思 淡斑精華內飄有棉絮物
data reading... 6508 id: 假美甲貼真的很美🎀🎀💅🏻🎀
data reading... 6509 id: 保養順序～
data 

data reading... 6675 id: #色號 嬌蘭24k新版色號00
data reading... 6676 id: #情報 GEO新品預購還可以抽獎
data reading... 6677 id: essence悠活曼谷經典眼彩盤
data reading... 6678 id: 韓國美妝產品對台灣的影響力調查
data reading... 6679 id: #分享 #面膜 我的美麗日記蜜若藍、LuLuLun ONE NIGHT
data reading... 6680 id: 貓奴終於收編PAUL&JOE😍
data reading... 6681 id: 比週年慶雙十一更划算（嬌蘭/碧兒泉）
data reading... 6682 id: 經典告別作Loreal X KARL聯名唇彩眼盤 必須都收🤞🏻
data reading... 6683 id: #隔離乳實測  SOFINA漾緁 水潤瓷效妝前隔離乳進化版
data reading... 6684 id: 💥卡友推爆！秋冬保濕大功臣〖ORBIS u芯生悠〗💧極光紀念組買2送5！一次體驗全系列～給你彈嫩光澤肌💖
data reading... 6685 id: 濃妝淡妝總相宜（？
data reading... 6686 id: 一年半空空賞【妝前.噴霧篇】
data reading... 6687 id: 肌研化妝水選擇
data reading... 6688 id: #終於等到你#資生堂香氛魔法盒
data reading... 6689 id: 分享 Missha 眼影 唇膏 組合 mini size
data reading... 6690 id: #週年慶 #文長 週年慶分享
data reading... 6691 id: Bobbi brown遮瑕筆
data reading... 6692 id: #問 耳洞發炎
data reading... 6693 id: Colourpop 水蜜桃盤眼妝分享
data reading... 6694 id: #問 統一時代百貨週年慶禮券用法
data reading... 6695 id: 原來兇手是B群....
data reading... 6696 id: #情報 Rimmel唇膏11元!
data reading... 6

data reading... 6864 id: 救救大乾肌
data reading... 6865 id: 媚比琳 摩天濃睫毛膏！
data reading... 6866 id: #請益 品木宣言 面膜術加減法
data reading... 6867 id: #試色 mini Nubian綠盤不專業眼妝分享
data reading... 6868 id: #情報 Rimmel粉底液附刷子299！
data reading... 6869 id: 質感試色推薦
data reading... 6870 id: 【開架腮紅】近期熱門腮紅大評比！
data reading... 6871 id: #混油實測🍂🍁superstay粉餅🤔
data reading... 6872 id: 霓淨思囤貨囤起來！
data reading... 6873 id: 求Maybelline精油唇膏15號類似色
data reading... 6874 id: ＃圖＃雅詩粉保濕粉底精華 少女水光肌VS乾燥厚粉後母肌
data reading... 6875 id: 寶拉10%B3毛孔調理分享
data reading... 6876 id: 屈臣氏的集點怎麼看(ಥ_ಥ)
data reading... 6877 id: 幫選韓系粉底液🙏
data reading... 6878 id: moilip 護唇膏真假貨
data reading... 6879 id: 眉毛求批
data reading... 6880 id: 冷門唇彩＃It's Skin Colorable Draw Tint唇釉分享
data reading... 6881 id: #情報 康是美1111特價超便宜！
data reading... 6882 id: 熱戀女子的簡單約會妝容分享🥰
data reading... 6883 id: 中壢/霧眉分享～超喜歡
data reading... 6884 id: #情報 理膚寶水B5修復霜限時特價
data reading... 6885 id: ＃大家近期都吃哪幾款B群呢？可以提供建議嗎？
data reading... 6886 id: 巨蛋美容展
data reading... 6887 id: 眼影盤選擇～～
data reading... 6888

data reading... 7053 id: ＃問 秋冬保濕粉底的選擇請益(有選手)
data reading... 7054 id: 蘭蔻分享文怎麼刪了QQ
data reading... 7055 id: ＃分享 DV的童齡霜
data reading... 7056 id: 多年愛用碧兒泉奇蹟水 鐵粉的＃使用解析
data reading... 7057 id: 專櫃粉底液選擇
data reading... 7058 id: 4U2 You heart me 與 Love me more 腮紅開箱分享~
data reading... 7059 id: #試色 essence新品crystal power九色眼影盤(11/9更)
data reading... 7060 id: #情報 優視達雙11優惠
data reading... 7061 id: #問 油痘肌水凝霜挑選
data reading... 7062 id: #週年慶開箱  ipsa/GA/mac/Ysl/Kiehl’s/NARS
data reading... 7063 id: MEMEBOX 贈品
data reading... 7064 id: #更/爸爸看到我說我皮膚怎麼那麼差
data reading... 7065 id: I’M MEME我愛柔霧磚土系唇膏 004白桃吐司土
data reading... 7066 id: 新手眼影選擇
data reading... 7067 id: #心得 蜜蠟除毛
data reading... 7068 id: suqqu粉霜vs嬌蘭小燈泡
data reading... 7069 id: 歐舒丹 護手霜+*(⁎⁍̴̛ᴗ⁍̴̛⁎)*+
data reading... 7070 id: 如何用楓葉眼影盤畫出仙氣感眼妝 韓國部落客妝容影片教學分享
data reading... 7071 id: 為什麼我會一直脫妝😭大家救救我吧謝謝
data reading... 7072 id: 燒你不燒錢
data reading... 7073 id: 雅詩蘭黛雙11獨家組合 讓我陷入膠著😣～～～～
data reading... 7074 id: #問 GA藍瓶粉底色號
data reading... 7075 id: #求滅火 #求試色sol

# ---------------正式版本分隔線---------------
- 測試版本代號：EGG_1.2 
- 開發人員： EGG Jerry Three


In [ ]:
# asb
import random
import time

url = "http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=99"
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df = df.append(Catch(rejs[0]['id']),ignore_index=True)   #0 - 29

delay_choices = [1]#, 3, 5, 7,20]  #延遲的秒數
delay = random.choice(delay_choices)  #隨機選取秒數

for j in range(10):
    last = str(int(df['id'].tail(1))) # 找出爬出資料的最後一筆ID
    url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=true&limit=99&before=" + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        if i % 101 == 0:
            print('第',j,'次執行，data reading...',i,'last',last,'chocice',delay)
    delay = random.choice(delay_choices)  #隨機選取秒數
    time.sleep(delay)  #延遲

df.to_csv('data/Op_Dcard_markeup.csv', index=False)
print(df.shape)
df

In [ ]:
# 熱門美妝版測試 (100筆)
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100'
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
for i in range(len(rejs)):
    df = df.append(Catch(rejs[i]['id']),ignore_index=True)   #0 - 29
    if i % 20 == 0:
        print('data reading...',i)
print(df.shape)
df.to_csv('data/Op_Dcard_markeup.csv', index=False)
df
# 99	235031315	#請益 #問 LA girl 遮瑕挑色
# 235066991

In [ ]:
#last = str(int(df['ID'].tail(1))) #last dt
#last

In [ ]:
# 透過迴圈讀取10*100篇文章，若需讀取更多資料，可以將range(10)中的數值提升
for j in range(10):
    last = str(int(df['ID'].tail(1))) # 找出爬出資料的最後一筆ID
    url = 'https://www.dcard.tw/_api/posts?popular=true&before=' + last
    #url = 'https://www.dcard.tw/_api/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        if i % 10 == 0:
            print('第',j,'次執行，data reading...',i)
            
print(df.shape)
df

In [ ]:
last = str(int(df['ID'].tail(1)))
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df

In [ ]:
# 熱門美妝版測試 (100筆)

for i in range(10):
    last = str(int(df['ID'].tail(1)))
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    df = pd.DataFrame()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']),ignore_index=True)
        if i % 20 == 0:
            print('data reading...',i)
    print(df.shape)

df.to_csv('data/Op_Dcard_markeup.csv', index=False)
df

# 99	235031315	#請益 #問 LA girl 遮瑕挑色

In [ ]:
last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID '235031315'
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
resq = requests.get(url)
rejs = resq.json()
for i in range(len(rejs)):
    df = df.append(Catch(rejs[i]['id']), ignore_index=True)
print(df.shape)


df

In [ ]:

#df

In [ ]:
##ok ^
for j in range(3):
    last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
print(df.shape)
#df

In [ ]:
# 透過迴圈讀取10*100篇文章，若需讀取更多資料，可以將range(10)中的數值提升
for j in range(10):
    last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        if i % 20 == 0:
            print('data reading...',i)
print(df.shape)
df


In [ ]:
df.to_excel('data/Dcard_markeup.excel', index=False)

欄位	說明	備註
ID	編號	
title	標題	
content	內容	
excerpt	摘要	
createdAt	發布時間	
updatedAt	更新時間	
commentCount	留言數	
forumName	分類	中文
forumAlias	分類	英文
gender	性別	
likeCount	心情數量	
reactions	心情細項	把以上心情細分為「愛心」、「哈哈」、「跪」、「森77」、「驚訝」等類型
topics	標籤	

In [ ]:
# 固定碼測試
def Catch_test(ID):
    url = 'https://www.dcard.tw/_api/posts/'+str(ID)
    requ = requests.get(url)
    rejs = requ.json()
    #rejs
    return(pd.DataFrame(
        data=
        [{'ID':rejs['id'],
          'title':rejs['title'],
          'content':rejs['content'],
          'excerpt':rejs['excerpt'],
          'createdAt':rejs['createdAt'],
          'updatedAt': rejs['updatedAt' ],
          'forumName':rejs['forumName'],
          'forumAlias':rejs['forumAlias'],
          'gender':rejs['gender'],
          'topics':rejs['topics']}],
        columns=['ID', 'title', 'content', 'excerpt', 'createdAt', 
                 'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
          )
Catch_test(235020702)